## Zadanie rekrutacyjne
Stwórz model uczenia maszynowego ktory będzie rozpoznawał czy tekst napisany jest w języku polskim czy angielskim. Dane uczące pobierz z dowolnego źródła. Metoda uczenia także jest dowolna. Nie korzystaj z gotowych funkcji rozpoznających języki.

W celu rozwiązania zadania pobrałem i przetworzyłem 2 książki "Władca Pierścieni" w wersji polskiej i angielskiej. Następnie za pomocą naiwnego klasyfikatora Bayesa stworzyłem model ropoznający język tekstu

In [1]:
import os
import pdfplumber
import pickle
import pandas as pd
import numpy as np
import re

In [4]:
def load_book(book_path):
    book = []
    with pdfplumber.open(book_path) as pdf:
        pages = pdf.pages
        for i,pg in enumerate(pages):
            try:
                text = pages[i].extract_text()
                text = text.strip()
                book.append(text)
            except Exception:
                pass
    return book

In [ ]:
# uwaga dużo obliczeń, jeśli nie chcesz czekać na wynik pomiń te komórkę

book_pl = load_book('LOTR_pl_all.pdf')
book_eng = load_book('LOTR_eng_all.pdf')

# zapis książek jako plik .pickle
with open(r'./pickle/book_pl.pickle', 'wb') as writer:
    pickle.dump(book_pl, writer)
with open( r'./pickle/book_eng.pickle', 'wb') as writer:
    pickle.dump(book_eng, writer)

In [ ]:
# wczytanie książek
book_pl =  pickle.load(open(r'./pickle/book_pl.pickle', 'rb'))
book_eng =  pickle.load(open(r'./pickle/book_eng.pickle', 'rb'))

In [ ]:
print(f'ilość stron pl: {len(book_pl)} \nilość stron eng: {len(book_eng)}')

In [ ]:
def words_list(book): 
    """Funkcja przetwarza wszystkie słowa z książki i dodaje je do jednej listy"""
    words_all = []
    for page in book:
        for words_page in page.split():
            words_all.append(words_page)
    return words_all

In [ ]:
words_all_pl = words_list(book_pl)
words_all_eng = words_list(book_eng)

In [ ]:
print('pl:', len(words_all_pl), '\neng:', len(words_all_eng)) #ilość słów

Angielska wersja ma więcej słów (około 10 000).

Tworzę Data Frame, każdy rząd to jedno słowo z etykietą języka.

In [ ]:
label_pl = np.array(['pl' for i in range(len(words_all_pl))])
label_eng = np.array(['eng' for i in range(len(words_all_eng))])
df_pl = pd.DataFrame(list(zip(words_all_pl, label_pl)), columns=['word', 'label'])
df_eng = pd.DataFrame(list(zip(words_all_eng, label_eng)), columns=['word', 'label'])
df_all = pd.concat([df_pl, df_eng], axis=0, ignore_index=True)

In [ ]:
df_all

In [ ]:
def clean_data(text):
    """operacje czyszczenia tekstu"""
    text = text.lower()
    text = re.sub(r'\W', '', text) # usunięcie znaków np. nawiasy
    text = re.sub(r'\d*', '',  text) # usuniecie liczb
    return text

df_all['word'] = df_all['word'].apply(clean_data)
df_all = df_all[df_all['word'] != '']

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_all['word'], df_all['label'],
                                                    test_size=0.2, random_state=42)

In [ ]:
# wytrenowanie modelu
pipeline = Pipeline([
    ('bow', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultinomialNB()),
])

pipeline.fit(X_train, y_train)

In [ ]:
predictions = pipeline.predict(X_test)

In [ ]:
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

Model radzi sobie całkiem dobrze. Confusion matrix wskazuje ile wartości zostało wskazanych poprawnie lub nie. Spośród wszystkich angielskich słow około 113 000* słów angielskich zostało poprawnie wskazanych jako angielskie, a tylko około 1 400 jako polskie.  
Trochę gorzej radzi sobie ze słowami polskimi (73 000 poprawnie, 16 000 nie). Model ma tendencję do wskazywania słów jako angielskie, gdy ma problem ze sklasyfikowaniem.  

*w zależności od losowych procesów, przy każdym trenowaniu podane dane mogą się delikatnie różnić

## Testowanie w praktyce

In [ ]:
text = 'We Francji dużo więcej zakażeń'
pipeline.predict_proba(text.split())

Słowo "We" w 91,8 % jest angielskie, w 8.2 % polskie.  
Słowo "Francji" w 56,1 % jest angielskie, w 43,9 % polskie.  
Słowo "Dużo" w 1,58 % jest angielskie, w 98,41 % polskie.  

Gdy model nie zna słowa zawsze podaje wartość 0.56 na angielski i 0.43 na polski. Być może jest to spowodowane większą ilością słów anglieskich w zbiorze treningowym. W funkcji poniżej zawarłem klauzulę, która pomija warotości w tym przedziale. Następnie dodaje prawdopodobieństwa poszczególnych słów, aby otrzymać całkowite prawdopodobieństwo języka.

In [ ]:
def language_detection(text):
    eng = 0
    pl = 0
    count = 0
    length = len(text.split())
    
    for i in range(length):
        score_eng = pipeline.predict_proba(text.split())[i][0]
        if score_eng < 0.57 and score_eng > 0.43:
            score_eng = 0
            count += 1
        score_pl = pipeline.predict_proba(text.split())[i][1]
        if score_pl < 0.57 and score_pl > 0.43:
            score_pl = 0
        eng += score_eng
        pl += score_pl
    
    if count == length:
        print('Did not recogize language')
    else:
        eng = round((eng / (length - count)) * 100, 2)
        pl = round((pl / (length - count)) * 100, 2)
        print(f'In {eng}% this is english \nIn {pl}% this is polish')

In [ ]:
text = 'I read the news today, oh boy About a lucky man who made the grade And though the news was rather sad Well'
language_detection(text)

In [ ]:
text_2 = 'We Francji dużo więcej zakażeń, ale mniej zgonów niż w Polsce. Dlaczego mamy tak wysoką śmiertelność?'
language_detection(text_2)

In [ ]:
text_3 = 'hello world, witaj świecie'
language_detection(text_3)

In [ ]:
text_4 = 'Jakub'
language_detection(text_4)